In [4]:
pip install --upgrade numexpr


   ---------------------------------------- 96.6/96.6 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: numexpr
    Found existing installation: numexpr 2.8.3
    Uninstalling numexpr-2.8.3:
      Successfully uninstalled numexpr-2.8.3
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.


In [1]:
import os
import pandas as pd

# Import counts

### Read data

In [2]:
#Read man counts
man_path = "01_Input/Manual_Counts"
man_data = []
man_loc = []

for file in os.listdir(man_path):
    #Read all excel files
    if file.endswith(".xlsx"):
        loc = file[0:3]
        data = pd.read_excel(os.path.join(man_path, file))
        data['Timestamp'] = data['Timestamp'].dt.floor('1s')
        #check if data for this loc already exists, then append or create new loc
        if loc in man_loc:
            index = man_loc.index(loc)
            man_data[index] = pd.concat([man_data[index], data], ignore_index=True)
        else:
            man_data.append(data)
            man_loc.append(loc)
        
#Read cam counts
cam_path = "01_Input/Camera_Counts"
cam_data = []
cam_loc = []

for file in os.listdir(cam_path):
    #Read all csv files
    if file.endswith(".csv"):
        loc = file[0:3]
        data = pd.read_csv(os.path.join(cam_path, file))
        data.rename(columns={"Unnamed: 0": "Timestamp"}, inplace=True)
        #Adjust format of timestamps
        data['Timestamp'] = pd.to_datetime(data['Timestamp'])
        data['Timestamp'] = data['Timestamp'].dt.tz_convert(None)
        #check if data for this loc already exists, then append or create new loc
        if loc in cam_loc:
            index = cam_loc.index(loc)
            cam_data[index] = pd.concat([cam_data[index], data], ignore_index=True)
        else:
            cam_data.append(data)
            cam_loc.append(loc)
            
#Read induction loop counts
inLoop_path = "01_Input/Induction_Loop_Counts"
inLoop_data = []
inLoop_loc = []

for file in os.listdir(inLoop_path):
    #Read all csv files
    if file.endswith(".csv"):
        loc = file[0:10]
        data = pd.read_csv(os.path.join(inLoop_path, file), encoding='latin1', sep=";")
        #Adjust format of timestamps
        data['Timestamp'] = pd.to_datetime(data['Timestamp'], format="%d.%m.%Y %H:%M")
        #check if data for this loc already exists, then append or create new loc
        if loc in inLoop_loc:
            index = inLoop_loc.index(loc)
            inLoop_data[index] = pd.concat([inLoop_data[index], data], ignore_index=True)
        else:
            inLoop_data.append(data)
            inLoop_loc.append(loc)

### Filter data

In [20]:
man_data_fil_hour = []

#filter to only have data from complete hours
for df in man_data:
    df['Date'] = df['Timestamp'].dt.date
    df['Hour'] = df['Timestamp'].dt.hour
    grouped = df.groupby(['Date', 'Hour'])
    counts = grouped.size()
    incomplete_indices = counts[counts != 6].index
    man_data_fil_hour.append(df[~df.set_index(['Date', 'Hour']).index.isin(incomplete_indices)])
    #df.drop(['Date', 'Hour'], axis=1, inplace=True)

In [21]:
#Filtered version of cam_data only containing timeframes available in man_data
cam_data_fil = [None] * len(cam_data)

#Fill the list
index_man = 0
for loc in man_loc:
    index_cam = cam_loc.index(loc)
    cam_data_fil[index_cam] = pd.merge(cam_data[index_cam], man_data[index_man]['Timestamp'], on='Timestamp', how='inner')
    index_man +=1

#Version of cam_data_fil only containing data where data for entire hour are available
cam_data_fil_hour = [None] * len(cam_data)

#Filtered version of inLoop_data only containing timeframes available in man_data_fil
inLoop_data_fil_hour = [None] * len(inLoop_data)

#Fill both lists
index_man = 0
for loc in man_loc:
    index_cam = cam_loc.index(loc)
    #index_inLoop = inLoop_loc.index(loc)
    cam_data_fil_hour[index_cam] = pd.merge(cam_data[index_cam], 
                                            man_data_fil_hour[index_man]['Timestamp'], on='Timestamp', how='inner')
    #inLoop_data_fil_hour[index_inLoop] = pd.merge(inLoop_data[index_inLoop], 
     #                                        man_data_fil_hour[index_man]['Timestamp'], on='Timestamp', how='inner')
    index_man +=1
    
print(cam_data_fil_hour[0])

Empty DataFrame
Columns: [Timestamp, Motorrad (R), LKW (S), LKW (R), Transporter (S), Kleintransporter (R), Kleintransporter (S), PKW (S), Fahrräder, Motorrad (S), PKW (R), Transporter (R)]
Index: []


# Comparisons

### Manual vs. Camera vs. Induction Loop

IndentationError: unexpected indent (3645202016.py, line 9)